In [1]:
# import relevant packages

import pandas as pd
import os
import re

from html import unescape
from sklearn.model_selection import train_test_split

pd.set_option('display.max_colwidth', None)

## Load raw datasets

In [21]:
df_dict = dict()

PATH = "/mounts/data/proj/faeze/data_efficient_hate/datasets/main/"

for file in sorted(os.listdir(PATH+'0_raw')):
    if 'measure' in file:
        continue
    if "ipynb" not in file:
        if file.endswith(".csv"):
            print(re.sub('\.csv$', '', file))
            df_dict[re.sub('\.csv$', '', file)] = pd.read_csv(f"{PATH}/0_raw/{file}", on_bad_lines='skip')
        else:
            print(re.sub('\.tsv$', '', file))
            df_dict[re.sub('\.tsv$', '', file)] = pd.read_csv(f"{PATH}/0_raw/{file}", on_bad_lines='skip', delimiter='\t')

bas19_es
dyn21_en
for19_pt
fou18_en
gahd24_de
has19_hi
has20_hi
has21_hi
implicit_en
ken20_en
ous19_ar
ous19_fr
san20_it
xdomain_en
xdomain_tr
xplain_en


## Reformat columns
Need separate logic for different datasets. 1 is for hateful, 0 for non-hateful.

In [22]:
df_dict["implicit_en"]['class'].replace({"implicit_hate":1, "explicit_hate":1, "not_hate":0}, inplace=True)
df_dict["implicit_en"].columns = ['text', 'label']


/tmp/ipykernel_1632883/3478913445.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dict["implicit_en"]['class'].replace({"implicit_hate":1, "explicit_hate":1, "not_hate":0}, inplace=True)
/tmp/ipykernel_1632883/3478913445.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_dict["implicit_en

In [23]:
df_dict["implicit_en"].label.value_counts()

label
0    13291
1     8189
Name: count, dtype: int64

In [24]:
# Dynabench 2021 / English
df_dict["dyn21_en"].label.replace({"hate":1, "nothate":0}, inplace=True)

# Founta 2018 / English
df_dict["fou18_en"].label.replace({'hateful': 1, "abusive": 0, "normal": 0, "spam": 0}, inplace = True)

# Kennedy 2020 / English
df_dict["ken20_en"].rename(columns={"label_hate_maj": "label"}, inplace=True)

# Fortuna 2019 / Portuguese
df_dict["for19_pt"].rename(columns={"hatespeech_comb": "label"}, inplace=True)

# Basile 2019 / Spanish
df_dict["bas19_es"].rename(columns={"HS": "label"}, inplace=True)

# Sanguinetti 2020 / Italian
df_dict["san20_it"].rename(columns={"hs": "label"}, inplace=True)

# Ousidhoum 2019 / Arabic & French
for d in ["ous19_ar", "ous19_fr"]:
    df_dict[d]["label"] = df_dict[d].sentiment.apply(lambda x: 1 if "hateful" in x else 0)
    # text was already cleaned in a way that conflicts with our later cleaning, so we align it here
    df_dict[d]["text"] = df_dict[d].tweet.apply(lambda x: x.replace("@url", "http"))

# HASOC 19, 20 and 21 / Hindi
for d in ["has19_hi", "has20_hi", "has21_hi"]:
    df_dict[d]["label"] = df_dict[d].task_2.apply(lambda x: 1 if x=="HATE" else 0)

# drop redundant columns
for dataset in df_dict:
    if "split" in df_dict[dataset].columns:
        df_dict[dataset] = df_dict[dataset][["text", "label", "split"]]
    else:
        df_dict[dataset] = df_dict[dataset][["text", "label"]]

/tmp/ipykernel_1632883/944236655.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dict["dyn21_en"].label.replace({"hate":1, "nothate":0}, inplace=True)
/tmp/ipykernel_1632883/944236655.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_dict["dyn21_en"].label.replace({"hate":1, "nothate":0}

## Clean text

In [25]:
def clean_text(text):
    text = unescape(text)
    text = re.sub(r"@[A-Za-z0-9_-]+",'@user',text) # format expected by XLM-T
    text = re.sub(r"http\S+",'http',text) # format expected by XLM-T
    text = re.sub(r"\n",' ',text)
    text = re.sub(r"\t",' ',text)
    text = text.replace("[URL]", "http") # format expected by XLM-T
    text = text.strip()
    return text

for dataset in df_dict:
    df_dict[dataset]["text"] = df_dict[dataset].text.apply(lambda x: clean_text(x))

In [26]:
# boost Kennedy 2020 / English to have 50% hate (up from ca. 30%)
df_dict["ken20_en"] = pd.concat([df_dict["ken20_en"][df_dict["ken20_en"].label==1], df_dict["ken20_en"][df_dict["ken20_en"].label==0].sample(11596, random_state=123)]).sample(frac=1, random_state=123)

# boost Founta 2018 / English to have 22% hate, which is max possible (up from ca. 5%)
df_dict["fou18_en"] = pd.concat([df_dict["fou18_en"][df_dict["fou18_en"].label==1], df_dict["fou18_en"][df_dict["fou18_en"].label==0].sample(17600, random_state=123)]).sample(frac=1, random_state=123)


In [27]:
df_dict["fou18_en"]["text"]

75975                                          RT @user: WE FUCKING DID IT!!!!!!!!!!!!!!!! ALL LOADED UP TO GO HOME TO FREEDOM!!!!!! http
28386                 @user Although one might argue that an attachment to free-floating commercialism may not be entirely unrelate… http
96446                             #ImpeachTrump #Trump #RussiaGate #Resist RT @user: Islamic State says U.S. 'being run by an idiot' http
54218                  @user The official gov stats show that the median earnings of FT female workers is 77 percent of FT male wor… http
98868                                                                               This shit way too crazy ayy! You do not amaze me ayy!
                                                                       ...                                                               
42289                                     My Twitter is worth $2,811.90!. Get your Twitter Value FREE at http #free #tools #freefollowers
41282          Takaharu for exampl

In [28]:
texts_list = []
texts_set = set()
for dataset in df_dict:
    for i in df_dict[dataset]["text"]:
        texts_list.append(i)
        if i in texts_set:
            print(dataset, i)
        texts_set.add(i)

len(texts_list), len(texts_set)

bas19_es @user @user Cállate perra
bas19_es @user @user Callate puta
bas19_es @user @user Cállate puta
bas19_es @user @user @user Cállate perra
bas19_es @user @user @user Cállate perra
bas19_es @user @user Cállate perra
bas19_es @user @user Cállate puta
bas19_es @user @user Callate hijo de perra
bas19_es @user @user Cállate perra
bas19_es @user @user @user @user Callate puta
bas19_es @user @user Cállate puta
bas19_es @user Cállate puta
bas19_es #NOME?
bas19_es @user Callate puta
bas19_es @user @user Callate puta
bas19_es @user Callate hija de puta
bas19_es @user Callate puta
bas19_es @user Cállate zorra
bas19_es @user cállate perra
bas19_es @user callate puta
bas19_es @user callate puta
bas19_es @user @user Cállate zorra
bas19_es @user Cállate perra
bas19_es @user Cállate perra
bas19_es @user Cállate zorra
bas19_es @user Cállate puta
bas19_es @user Cállate puta
bas19_es @user @user Cállate puta barata
bas19_es @user Cállate puta
bas19_es @user @user Callate puta
bas19_es @user Cállate 

(276931, 273794)

## Show descriptive stats

In [10]:
def descriptive_stats(df):
    n_total = df.shape[0]
    n_hate = df.label.sum()
    print("{} entries, of which {} ({:.2%}) are hateful.".format(n_total, n_hate, n_hate/n_total))
    return df.label.sum()/len(df), len(df)

for dataset in df_dict:
    print(dataset.upper())
    descriptive_stats(df_dict[dataset])
    display(df_dict[dataset].sample(5))
    print()

BAS19_ES
6600 entries, of which 2739 (41.50%) are hateful.


,text,label
184,// creo que algum vecino esta escuchando reggaeton pero no lo tengo claro porque suena como si coges una música árabe y le metes autotune rollo robótico,0
3772,Otro montaje sobre los refugiados ahogándose en el mar http,0
1949,"Arabe es claveee, dio me vuelvo locaa me encanta! @user",0
1916,"""El año pasado me teñí de rubia para ir a la CDMX y no indignarme porque me dijeran """"güera"""". http",0
1676,"a VER SI ES VERDAD Y, dejando de ser un bulo, se impone un poco de cordura en Europa . http",0



DYN21_EN
41144 entries, of which 22175 (53.90%) are hateful.


,text,label,split
15009,I whish I had been there only to enjoy the smell of the burning LGBT flag!!!,1,train
201,the NHS is rubbish,0,dev
23117,I actually believe religion is about controlling women. Men can gain a personal servant and they believe they are entitled to that.,0,dev
2520,I want to fucking kill myself because I live next to a tranny,1,train
17242,"The immigration rules violate personal liberty, insult your intelligence, and are symbol of ignorance and tyranny. I have not spoken to one chinese person since the start of this plandemic and have tried to encourage others to be brave enough not to comply. Unfortunately, most people are cowards. I quit my job and stopped doing business at a number of places which either insulted me by asking me to not voice my opinion one or in a few cases denied me service. Can someone explain why it is so hard to get the average person to grow some balls?""",1,train



FOR19_PT
5670 entries, of which 1788 (31.53%) are hateful.


...

,text,label,split
26496,"Anytime I remember Gareth Bale is stuck on the Madrid bench I feel real pain man, was so so good and is now the real madrid equivalent of Ozil ffs http",0,train
45571,HAHAHAHAHA Turks are so mad i love to see it 🤣🤣🤣🤣🤣 they were saying they will destroy Latvia 🤣🤣🤣,1,test
17313,Joe Biden under probe in Ukraine for alleged link to top prosecutor’s 2016 ouster: report http #FoxNews,0,train
47499,YEP OFF TWITTER UNTIL 7 THIS BOUT TO GET OUTTA HAND IMMA EAT THIS STUFFED CRUST PIZZA AND NAP THEN HOPEFULLY THIS GENDER WAR WILL HAVE DIED DOWN,0,test
36387,Just tryna be thicc like Nic #body #bodybuilding #bigbutt #LGBTQ http,0,validation



XDOMAIN_TR
45159 entries, of which 19269 (42.67%) are hateful.


,text,label,split
18156,"Mazinde bin şike yatar, geber fenerbahçe. Ne mutlu seni yenene, geber fenerbahçe #KadıkoeydeKaraGece",1,train
38754,"Bugün çok acaip haberler okudum, Adam imam nikahlı karısının kapısına bomba düzeneği kurmuş 😳 Başka bir adam başka bir hanımı 1570 farklı numaradan taciz etmiş. 😳",0,test
42311,LGBT pisliği yüzünden Gökkuşağından nefret ettim,1,test
43910,Ruslar dünyaya duyurdu! Bayraktarlara karşı bir süper silah... @user http,0,test
25912,KAZA DEĞİL CİNAYET! İstanbul’da 3 farklı geriye düşmesine rağmen muhteşem bir geri dönüş sergileyerek skoru dengeleyen Bursaspor’da coşku hakemin hatalı kararları nedeniyle camiada yaşanan tepkinin gölgesinde kaldı. http http,1,train



XPLAIN_EN
13749 entries, of which 7814 (56.83%) are hateful.


,text,label,split
9345,<user> <user> we are iraqi refugees we ask you to process our delayed files please standwithiraqirefugees,1,train
10010,the below countries adamantly refused syrian refugees and their reasons were exactly the same they are way too dangerous to allow in libtards are putting themselves especially their children in extreme danger by supporting them,1,train
13304,<user> do not understand why in america it all about black or white there are yellow mongoloids and brown orientals people as well most suppressed ones never get to raise their voice v unbeatable loosing in agt is an example to show how racially biased americans are,1,test
687,i am an ashkenazi jew,1,train
1539,<user> gay asf of u to say this kisses u,1,train


In [82]:
new_dict = {}
# new_dict["xdomain_tr"] = df_dict["xdomain_tr"]
# new_dict["xdomain_en"] = df_dict["xdomain_en"]
# new_dict["gahd24_de"] = df_dict["gahd24_de"]
new_dict["xplain_en"] = df_dict["xplain_en"]
# new_dict["measure_en"] = df_dict["measure_en"]
df_dict = new_dict

In [83]:
df_dict.keys()

dict_keys(['xplain_en'])

In [84]:
df = new_dict['xplain_en']

In [85]:
df.split.value_counts()

split
train         10999
test           1376
validation     1374
Name: count, dtype: int64

In [89]:
train_df = df[df['split'] == 'train']
dev_df = df[df['split'] == 'validation']
test_df = df[df['split'] == 'test']
# Adjust dev and test sizes
dev_to_test = dev_df.sample(n=2000 - len(test_df), random_state=123)  # Sample 354 from dev to add to test
remaining_dev = dev_df.drop(dev_to_test.index)

# Create new test set by combining existing test and sampled dev
new_test_df = pd.concat([test_df, dev_to_test])

# Split the remaining dev into dev (500 instances) and extra_dev
new_dev_df = remaining_dev.sample(n=500, random_state=123)
extra_dev_df = remaining_dev.drop(new_dev_df.index)

train_df = train_df.drop(columns=['split'])
new_dev_df = new_dev_df.drop(columns=['split'])
new_test_df = new_test_df.drop(columns=['split'])
extra_dev_df = extra_dev_df.drop(columns=['split'])

In [101]:
path = '/mounts/work/faeze/data_efficient_hate/datasets/main/1_clean/xplain_en/'
df_dict['xplain_en'] = train_df
# Save the adjusted splits to separate files
train_df.to_csv(path+f'train_{train_df.shape[0]}.csv', index=False)
new_dev_df.to_csv(path + f'dev_{new_dev_df.shape[0]}.csv', index=False)
new_test_df.to_csv(path + f'test_{new_test_df.shape[0]}.csv', index=False)
extra_dev_df.to_csv(path + f'extra_dev_{extra_dev_df.shape[0]}.csv', index=False)


In [102]:
train_df.shape, new_dev_df.shape, extra_dev_df.shape, new_test_df.shape

((10999, 2), (500, 2), (250, 2), (2000, 2))

(7701, 2)

In [7]:
# set aside 2k from each dataset for testing and 500 for dev
# except for Ousidhoum in French and Arabic, where train set would otherwise be too small
# and for HASOC 20 and 21 in Hindi, where test splits are given

TEST_SIZE = 2000
DEV_SIZE = 500

for dataset in df_dict:
    if "ous19_fr" in dataset:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = 2000, random_state=123)
        devset, testset = train_test_split(devtest, test_size = 1500, random_state=123)
        devset.to_csv(PATH + f"/1_clean/{dataset}/dev_500.csv", index=False)
        testset.to_csv(PATH + f"/1_clean/{dataset}/test_1500.csv", index=False)
    elif "ous19_ar" in dataset:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = 1300, random_state=123)
        devset, testset = train_test_split(devtest, test_size = 1000, random_state=123)
        devset.to_csv(PATH + f"/1_clean/{dataset}/dev_300.csv", index=False)
        testset.to_csv(PATH + f"/1_clean/{dataset}/test_1000.csv", index=False)
    elif "has19_hi" in dataset or "has20_hi" in dataset: # use provided test sets
        df_dict[dataset][df_dict[dataset]["split"]=="test"].to_csv(PATH + f"/1_clean/{dataset}/test_{len(df_dict[dataset][df_dict[dataset]['split']=='test'])}.csv", index=False)
        df_dict[dataset], devset = train_test_split(df_dict[dataset][df_dict[dataset]["split"]=="train"], test_size = 500, random_state=123)
        devset.to_csv(PATH + f"/1_clean/{dataset}/dev_500.csv", index=False)
    else:
        df_dict[dataset], devtest = train_test_split(df_dict[dataset], test_size = TEST_SIZE+DEV_SIZE, random_state=123)
        devset, testset = train_test_split(devtest, test_size = TEST_SIZE, random_state=123)
        devset.to_csv(PATH + f"/1_clean/{dataset}/dev_{DEV_SIZE}.csv", index=False)
        testset.to_csv(PATH + f"/1_clean/{dataset}/test_{TEST_SIZE}.csv", index=False)
        
# export all data that is not test or dev, so we can use it for full sample training
for dataset in df_dict:
    df_dict[dataset].to_csv(PATH + f"/1_clean/{dataset}/train_{len(df_dict[dataset])}.csv", index=False)

In [8]:
# create differently-sized train portions from rest of data

SEEDS = 10 # for repeated experiments with different random data selection
N_RANGE = [10, 20, 30, 40, 50, 100, 200, 300, 400, 500, 1000, 2000, 3000, 4000, 5000, 10000, 20000]

for dataset in df_dict:
    print(dataset.upper())
    for n in N_RANGE:
        
        # save all splits for English test sets
        if n<len(df_dict[dataset]) and ("_en" in dataset):
            print(f"  saving n = {n} training set")
            for random_state in range(1, SEEDS+1):
                df_dict[dataset].sample(n, random_state = random_state).to_csv(PATH + f"/1_clean/{dataset}/train/train_{n}_rs{random_state}.csv",index=False)
        
        # save splits up to 2k for other datasets
        elif n<len(df_dict[dataset]) and n<=2000: 
            print(f"  saving n = {n} training set")
            for random_state in range(1, SEEDS+1):  
                df_dict[dataset].sample(n, random_state = random_state).to_csv(PATH + f"/1_clean/{dataset}/train/train_{n}_rs{random_state}.csv",index=False)
    
    print()

IMPLICIT_EN
  saving n = 10 training set
  saving n = 20 training set
  saving n = 30 training set
  saving n = 40 training set
  saving n = 50 training set
  saving n = 100 training set
  saving n = 200 training set
  saving n = 300 training set
  saving n = 400 training set
  saving n = 500 training set
  saving n = 1000 training set
  saving n = 2000 training set
  saving n = 3000 training set
  saving n = 4000 training set
  saving n = 5000 training set
  saving n = 10000 training set



In [9]:
df_dict.keys()

dict_keys(['implicit_en'])